# Hierarchical Agglomerative Clustering Implementation

### Loading Dependencies

In [74]:
import numpy as np
import pickle
import time
import sys
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *
from sklearn.decomposition import PCA as sklearnPCA
init_notebook_mode(connected=True) #Set jupyter notebook mode to true for running pyplot

### Preprocess the input data

**Input Parameters** : file name


**returns**: output matrix X, disease id and the ground truth

In [75]:
def preprocess(filename):
    inpdata = np.genfromtxt(filename,delimiter = '\t')
    X = np.loadtxt(filename,delimiter = '\t', usecols = range(2, inpdata.shape[1]), dtype = 'S15')
    gen_id = np.loadtxt(filename,delimiter = '\t', usecols = 0, dtype = 'S15')
    ground_truth = np.loadtxt(filename,delimiter = '\t', usecols = 1, dtype = 'S15')
    return X, gen_id, ground_truth

## Function to run the Heirarchical Agglomerative Clustering Algorithm

**Input Parameters** : input matrix X, disease id and the number of clusters k.


**returns**: cluster pca matrix and clusters

In [76]:
def agglomerative_clus(gen_id, distance_matrix, k):
    clusters = gen_id.tolist()
    cluster_pca = []
    for i in gen_id:
        cluster_pca.append([i])
    while(len(clusters) > k):
        in1, in2 = find_min_indices(distance_matrix)
        left = -1
        right = -1
        if(in1 < in2):
            left = in1
            right = in2
        else:
            right = in1 
            left = in2
        if(type(clusters[left]) == bytes):
            clusters[left] = clusters[left].decode("utf-8") 
        if(type(clusters[right]) == bytes):
            clusters[right] = clusters[right].decode("utf-8") 
        merged_cluster = "("+clusters[left] + "," + clusters[right] + ")"
        clusters[left] = merged_cluster
        cluster_pca[left] = cluster_pca[left] + cluster_pca[right]
        del clusters[right]
        del cluster_pca[right]
        for i in range(0, len(distance_matrix)):
            distance_matrix[left][i] = min(distance_matrix[left][i], distance_matrix[right][i])
            distance_matrix[i][left] = min(distance_matrix[left][i], distance_matrix[right][i])
        distance_matrix = np.delete(distance_matrix, right, 0)
        distance_matrix = np.delete(distance_matrix, right, 1)
    return cluster_pca, clusters

### Draw Scatter Plot using plotly

**Input Parameters** : 2 dimentional data and its label


**prints**: visualized clusters

In [77]:
def draw_scatter_plot(Y, labels):
    unique_labels = set(labels)
    points = []
    for name in unique_labels:
        x = []
        y = []
        for i in range(0, len(labels)):
            if(labels[i] == name):
                x.append(Y[i,0])
                y.append(Y[i,1])
        x = np.array(x)
        y = np.array(y)
        point = Scatter(
            x = x,
            y = y,
            mode='markers',
            name = int(name),
            marker=Marker(size=12, line=Line(color='rgba(217, 154, 217, 123)',width=0.5),opacity=0.9))
        points.append(point)
    data = Data(points)
    layout = Layout(xaxis=XAxis(title='Principle Component 1', showline=True),
                    yaxis=YAxis(title='Principle Component 2', showline=True))
    fig = Figure(data=data, layout=layout)
    iplot(fig)

### Find the indices of cell which has the minimum value for HAC 

**Input Parameters** : distance matrix of size m X n


**returns**: indices i and j

In [78]:
def find_min_indices(distance_matrix):
    index1 = -1
    index2 = -1
    min_elem = sys.maxsize
    for i in range(0, distance_matrix.shape[0]):
        for j in range(0, distance_matrix.shape[1]):
            if(i != j):
                if(distance_matrix[i][j] < min_elem):
                    min_elem = distance_matrix[i][j]
                    index1 = i 
                    index2 = j
    return index1, index2

### Calculate the Jaccard value of predicted data

**Input Parameters** : actual ground truth matrix and predicted ground truth


**returns**: None


**prints**: Jaccard and Rand value on console

In [79]:
def calculateJackard(actual_ground_truth, predicted_ground_truth):
    m00, m01, m10, m11 = 0, 0, 0, 0
    for i in range(0, len(actual_ground_truth)):
        for j in range(0, len(actual_ground_truth)):
            if((actual_ground_truth[i] != actual_ground_truth[j]) and (predicted_ground_truth[i] != predicted_ground_truth[j])):
                m00 += 1
            elif((actual_ground_truth[i] == actual_ground_truth[j]) and (predicted_ground_truth[i] != predicted_ground_truth[j])):
                m01 += 1
            elif((actual_ground_truth[i] != actual_ground_truth[j]) and (predicted_ground_truth[i] == predicted_ground_truth[j])):
                m10 += 1
            elif((actual_ground_truth[i] == actual_ground_truth[j]) and (predicted_ground_truth[i] == predicted_ground_truth[j])):
                m11 += 1
    jaccard = m11 / float(m11 + m10 + m01)
    rand = (m11 + m00) / float(m11 + m10 + m01 + m00)
    print(" Jaccard is : " + str(jaccard)),
    print(" Rand is : " + str(rand))

### Run principle component analysis to convert n-dimensional data to 2 dimensions in order to visualize
**Input Parameters** : input matrix X


**returns**: Eigen Vector Y

In [80]:
def runPCA(X):
    sklearn_pca = sklearnPCA(n_components=2)
    Y_sklearn = sklearn_pca.fit_transform(X)
    return Y_sklearn

## Driver program to run the above code

**Input Parameters** : file name and no of clusters


**prints**: Jackard value and scatter plot

In [81]:
def driver(file_name, k_value):
    X, gen_id, ground_truth = preprocess(file_name)
    len_X = X.shape[0]
    distance_matrix = np.zeros((len_X, len_X), dtype='float64')  
    for i in range(0, len_X):
        for j in range(0, len_X):
            if(i != j):
                dis = 0
                for k in range(0, X.shape[1]):
                    dis = dis + np.square(np.subtract(float(X[i][k]), float(X[j][k])))
                distance_matrix[i][j] = np.sqrt(dis) 
                distance_matrix[j][i] = np.sqrt(dis)
    start = time.time()
    clusters_pca, clusters = agglomerative_clus(gen_id, distance_matrix, k_value)
    heirarical_ground_truth = [0]*len(gen_id)
    for i in range(0, len(clusters_pca)):
        for j in clusters_pca[i]:
            heirarical_ground_truth[int(j)-1] = i
    print("Time is : "),
    print("--- %s seconds ---" % (time.time() - start))
    calculateJackard(ground_truth, heirarical_ground_truth)
    Y_pca = runPCA(X)
    draw_scatter_plot(Y_pca, heirarical_ground_truth)

## To run this algorithm for a different file or no of clusters, please change parameters here

In [82]:
file_name = "data/iyer.txt"
k_value = 3
driver(file_name, k_value)

Time is : 
--- 22.018948078155518 seconds ---
 Jaccard is : 0.1559309385706048
 Rand is : 0.1621204015129691


In [85]:
file_name = "data/cho.txt"
k_value = 5
driver(file_name, k_value)

Time is : 
--- 9.249238967895508 seconds ---
 Jaccard is : 0.22839497757358454
 Rand is : 0.24027490670890495


In [87]:
file_name = "data/new_dataset_2.txt"
k_value = 2
driver(file_name, k_value)

Time is : 
--- 0.0006818771362304688 seconds ---
 Jaccard is : 1.0
 Rand is : 1.0
